# R: Life Modelling Recipes

By Pat Reen - originally published on his GitHub site [here](https://pat-reen.github.io/Modelling-Recipes/).
<!-- Add icon library -->
<link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/4.7.0/css/font-awesome.min.css">

<!-- Add icon font -->
<p style="position:absolute; top:0; right:20px; ">
    <a href="https://www.linkedin.com/in/patrick-reen/" target="_blank" class="fa fa-linkedin" style="font-size:24px"></a>
    <a href="https://github.com/Pat-Reen/" target="_blank" class="fa fa-github" style="font-size:24px"></a>
</p>

# Overview
## Background 
This document sets out a few practical recipes for modelling with (life) insurance data. Insurance events are typically of low probability - these recipes consider some of the limitations of "small data" model fitting (where the observations of interest are sparse) and other topics for insurance like comparisons to standard tables. Themes

* Common data transforms, summary stats, and simple visualisations
* Regression 
  + Grouped vs ungrouped data
  + Choice of: response distribution, link (and offsets), explanatory variables 
  + Modelling variance to industry/ reference (A/E or A - E)
  + Model selection: stepwise regression, likelihood tests, model evaluation
  + Predictions, confidence intervals and visualisations
* Bayesian regression and other classification models - to follow.

See link to [GitHub repository](https://github.com/Pat-Reen/Modelling-Recipes) which has the [detailed code](https://github.com/Pat-Reen/Modelling-Recipes/blob/main/RecipeBook.Rmd).

## Libraries
A list of packages used in the recipe book.

In [1]:
library(rmdformats) # theme for the HTML doc
library(bookdown)   # bibliography formatting
library(kableExtra) # formatting tables
library(scales)     # data formatting  
library(dplyr)      # tidyverse: data manipulation
library(tidyr)      # tidyverse: tidy messy data
library(corrplot)   # correlation matrix visualisation, optional
library(ggplot2)    # tidyverse: graphs
library(pdp)        # tidyverse: arranging plots
library(GGally)     # tidyverse: extension of ggplot2
library(ggthemes)   # tidyverse: additional themes for ggplot, optional
library(plotly)     # graphs, including 3D 
library(caret)      # sampling techniques
library(broom)      # tidyverse: visualising statistical objects
library(pROC)       # visualising ROC curves
library(lmtest)     # tests for regression models

# packages below have some interaction with earlier packages, not always needed
library(arm)        # binned residual plot
library(msme)       # statistical tests, pearson dispersion
library(MASS)       # statistics



Attaching package: ‘dplyr’


The following object is masked from ‘package:kableExtra’:

    group_rows


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


corrplot 0.90 loaded

Registered S3 method overwritten by 'GGally':
  method from   
  +.gg   ggplot2


Attaching package: ‘plotly’


The following object is masked from ‘package:ggplot2’:

    last_plot


The following object is masked from ‘package:stats’:

    filter


The following object is masked from ‘package:graphics’:

    layout


Loading required package: lattice

Type 'citation("pROC")' for a citation.


Attaching package: ‘pROC’


The following objects are masked from ‘package:stats’:

    cov, smooth, var


Loading required package: zoo


Attaching package: ‘zoo’


The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric


Loading required package: MASS


Attaching package:

In [2]:
ifrm <- function(x, env = globalenv()) 
{
  if(exists(x, envir = env)) 
  {
    rm(list = x, envir = env)
  }
}

## Further reading
A few books and articles of interest:

* [R Markdown Cookbook](https://bookdown.org/yihui/rmarkdown-cookbook/) has everything you need to know to set up an r markdown document like this one.
* [Generalised Linear Models for Insurance Data](https://www.cambridge.org/au/academic/subjects/statistics-probability/statistics-econometrics-finance-and-insurance/generalized-linear-models-insurance-data?format=HB&isbn=9780521879149) is a great book introducing GLMs in the context of insurance, considering problems specific to insurance data.
* [Tidyverse documentation](https://www.tidyverse.org/) full set of documentation for Tidyverse packages (packages for data science) e.g. dplyr for data manipulation; tidyr for tidying up messy data; ggplot for visualisation.

# Data Simulation
This section sets out a method for generating dummy data. The simulated data is intended to reflect typical data used in an analysis of disability income incidence experience and is used throughout this analysis. Replace this data with your actual data.

More detail on the techniques used can be found in the section on data manipulation.

## Simulating policies
We start by simulating a mix of 200k policies over 3 years. Some simplifying assumptions e.g. nil lapse/ new bus (no allowance for part years of exposure), no indexation. Mix of business assumptions for benefit period, waiting period and occupation taken  taken from [@BusMix], with the remainder based on an anecdotal view of industry mix not intended to be reflective of any one business. 

In [3]:
# set the seed value (for the random number generator) so that the simulated data frame can be replicated later
set.seed(10)
# create 200k policies
n <- 200000

# data frame columns
# policy_year skewed to early years, but tail is fat
df <- data.frame(id = c(1:n), cal_year = 2018,policy_year = round(rweibull(n, scale=5, shape=2),0)) 
df <- df %>% mutate(sex = replicate(n,sample(c("m","f","u"), size=1, replace=TRUE, prob=c(.75,.20,.05))),
smoker = replicate(n,sample(c("n","s","u"), size=1, replace=TRUE, prob=c(.85,.1,.05))),
# mix of business for benefit_period, waiting_period, occupation taken from industry presentation
benefit_period = replicate(n,sample(c("a65","2yr","5yr"), size=1, replace=TRUE, prob=c(.76,.12,.12))),
waiting_period = replicate(n,sample(c("14d","30d","90d","720d"), size=1, replace=TRUE, prob=c(.04,.7,.15,.11))),
occupation = replicate(n,sample(c("prof","sed","techn","blue","white"), size=1, replace=TRUE, prob=c(.4,.2,.2,.1,.1))),
# age and policy year correlated; age normally distributed around 40 + policy_year (where policy_year is distributed around 5 years), floored at 25, capped at 60
age = round(pmax(pmin(rnorm(n,mean = 40+policy_year, sd = 5),60),25),0),
# sum_assured, age and occupation are correlated; sum assured normally distributed around some mean (dependent on age rounded to 10 and on occupation), floored at 500
sum_assured = 
  round(
    pmax(
      rnorm(n,mean = (round(age,-1)*100+ 1000) * 
      case_when(occupation %in% c("white","prof") ~ 1.5, occupation %in% c("sed") ~ 1.3 , TRUE ~ 1), 
      sd = 2000),500),
      0)
  )
# generate 3 years of exposure for the 200k policies => assume no lapses or new business
df2 <- df %>% mutate(cal_year=cal_year+1,policy_year=policy_year+1,age=age+1)
df3 <- df2 %>% mutate(cal_year=cal_year+1,policy_year=policy_year+1,age=age+1)
df <- rbind(df,df2,df3)


## Expected claim rate 
Set p values from which to simulate claims. The crude p values below were derived from the Society of Actuaries Analysis of USA Individual Disability Claim Incidence Experience from 2006 to 2014 [@SOA_study], with some allowance for Australian industry differentials [@BusMix_FSC].

In [4]:
# by cause, age and sex, based upon polynomials fitted to crude actual rates
# sickness
f_sick_age_m <- function(age) {-0.0000003*age^3 + 0.000047*age^2 - 0.00203*age + 0.02715}
f_sick_age_f <- function(age) {-0.0000002*age^3 + 0.000026*age^2 - 0.00107*age + 0.01550} 	  	 	  
f_sick_age_u <- function(age) {f_sick_age_f(age)*1.2}
f_sick_age   <- function(age,sex) {case_when(sex == "m" ~ f_sick_age_m(age), sex == "f" ~ f_sick_age_f(age), sex == "u" ~ f_sick_age_u(age))}

# accident
f_acc_age_m <- function(age) {-0.00000002*age^3 + 0.000004*age^2 - 0.00020*age + 0.00340}
f_acc_age_f <- function(age) {-0.00000004*age^3 + 0.000007*age^2 - 0.00027*age + 0.00374} 	  	 	  
f_acc_age_u <- function(age) {f_sick_age_f(age)*1.2}
f_acc_age   <- function(age,sex) {case_when(sex == "m" ~ f_acc_age_m(age), sex == "f" ~ f_acc_age_f(age), sex == "u" ~ f_acc_age_u(age))}

# smoker, wp and occ based upon ratio of crude actual rates by category
# occupation adjustment informed by FSC commentary on DI incidence experience
f_smoker   <- function(smoker) {case_when(smoker == "n" ~ 1, smoker == "s" ~ 1.45, smoker == "u" ~ 0.9)}
f_wp   <- function(waiting_period) {case_when(waiting_period == "14d" ~ 1.4, waiting_period == "30d" ~ 1, waiting_period == "90d" ~ 0.3, waiting_period == "720d" ~ 0.2)}
f_occ_sick   <- function(occupation) {case_when(occupation == "prof" ~ 1, occupation == "sed" ~ 1, occupation == "techn" ~ 1, occupation == "blue" ~ 1, occupation == "white" ~ 1)}
f_occ_acc   <- function(occupation) {case_when(occupation == "prof" ~ 1, occupation == "sed" ~ 1, occupation == "techn" ~ 4.5, occupation == "blue" ~ 4.5, occupation == "white" ~ 1)}

# anecdotal allowance for higher rates at larger policy size and for older policies
f_sa_sick <- function(sum_assured) {case_when(sum_assured<=6000 ~ 1, sum_assured>6000 & sum_assured<=10000 ~ 1.1, sum_assured>10000 ~ 1.3)}
f_sa_acc <- function(sum_assured) {case_when(sum_assured<=6000 ~ 1, sum_assured>6000 & sum_assured<=10000 ~ 1, sum_assured>10000 ~ 1)}
f_pol_yr_sick <- function(policy_year) {case_when(policy_year<=5 ~ 1, policy_year>5 & policy_year<=10 ~ 1.1, policy_year>10 ~ 1.3)}
f_pol_yr_acc <- function(policy_year) {case_when(policy_year<=5 ~ 1, policy_year>5 & policy_year<=10 ~ 1, policy_year>10 ~ 1)}

## Expected claims
Add the crude p values to the data and simulate 1 draw from a binomial with prob = p for each record. Gives us a vector of claim/no-claim for each policy. Some simplifying assumptions like independence of sample across years for each policy and independence of accident and sickness incidences.

In [5]:
# add crude expected
df$inc_sick_expected=f_sick_age(df$age,df$sex)*f_smoker(df$smoker)*f_wp(df$waiting_period)*f_occ_sick(df$occupation)*f_sa_sick(df$sum_assured)*f_pol_yr_sick(df$policy_year)
df$inc_acc_expected=f_acc_age(df$age,df$sex)*f_smoker(df$smoker)*f_wp(df$waiting_period)*f_occ_acc(df$occupation)*f_sa_acc(df$sum_assured)*f_pol_yr_acc(df$policy_year)
# add prediction
df$inc_count_sick = sapply(df$inc_sick_expected,function(z){rbinom(1,1,z)})
df$inc_count_acc = sapply(df$inc_acc_expected,function(z){rbinom(1,1,z)})*(1-df$inc_count_sick)
df$inc_count_tot = df$inc_count_sick + df$inc_count_acc
# add amounts prediction
df$inc_amount_sick = df$inc_count_sick * df$sum_assured
df$inc_amount_acc =  df$inc_count_acc * df$sum_assured
df$inc_amount_tot =  df$inc_count_tot * df$sum_assured

## Grouped data

The data generated above are records for each individual policy, however data like this is often grouped as it is easier to store and computation is easier [@GLM_Insurance, p49, 105]. Later we will consider the differences between a model on ungrouped vs grouped data.

In [6]:
# group data (see section on data manipulation below)
df_grp <- df %>% group_by(cal_year, policy_year, sex, smoker, benefit_period, waiting_period, occupation, age) %>% 
summarise(sum_assured=sum(sum_assured),inc_count_sick_exp=sum(inc_sick_expected),inc_count_acc_exp=sum(inc_acc_expected),        inc_count_sick=sum(inc_count_sick),inc_count_acc=sum(inc_count_acc),inc_count_tot=sum(inc_count_tot),inc_amount_sick=sum(inc_amount_sick),inc_amount_acc=sum(inc_amount_acc),inc_amount_tot=sum(inc_amount_tot), exposure=n(),.groups = 'drop') 


Check that the exposure for the grouped data is the same as the total on ungrouped:

In [7]:
# check count - same as total row count of the main df
sum(df_grp$exposure)

[1] 600000

And that the number of rows of data are significantly lower:

In [8]:
# number of rows of the grouped data is significantly lower
nrow(df_grp)

[1] 109590

# Data Exploration

The sections below rely heavily upon the dplyr package. 

## Data structure
Looking at the metadata for the data frame and a sample of the contents.

In [9]:
# MASS package interferes with select()
select <- dplyr::select

In [10]:
# glimpse() or str() returns detail on the structure of the data frame. Our data consists of 600k rows and 15 columns. The columns are policy ID, several explanatory variables like sex and smoker, expected counts of claim (inc_sick_expected and inc_acc_expected) and actual counts of claim (inc_count_sick/acc/tot).
glimpse(df)

Rows: 600,000
Columns: 18
$ id                <int> 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 1…
$ cal_year          <dbl> 2018, 2018, 2018, 2018, 2018, 2018, 2018, 2018, 2018…
$ policy_year       <dbl> 4, 5, 5, 3, 8, 6, 6, 6, 3, 5, 3, 4, 7, 4, 5, 5, 9, 6…
$ sex               <chr> "m", "f", "m", "m", "f", "f", "m", "m", "m", "m", "m…
$ smoker            <chr> "n", "n", "n", "n", "n", "n", "n", "n", "s", "n", "n…
$ benefit_period    <chr> "a65", "5yr", "a65", "a65", "a65", "2yr", "a65", "a6…
$ waiting_period    <chr> "90d", "30d", "30d", "30d", "30d", "14d", "720d", "3…
$ occupation        <chr> "techn", "blue", "blue", "prof", "sed", "prof", "tec…
$ age               <dbl> 41, 46, 41, 27, 53, 49, 54, 42, 43, 52, 36, 47, 47, …
$ sum_assured       <dbl> 7119, 5582, 6113, 5147, 8864, 11209, 6378, 6780, 597…
$ inc_sick_expected <dbl> 0.000742731, 0.001828800, 0.002475770, 0.000698100, …
$ inc_acc_expected  <dbl> 0.0007365330, 0.0100735200, 0.0024551100, 0.00052234…
$ inc_count_si

In [11]:
# head() returns the first 6 rows of the data frame. Similar to head(), sample_n() returns rows from our data frame, however these are chosen randomly. e.g. sample_n(df,5,replace=FALSE)
head(df)

,id,cal_year,policy_year,sex,smoker,benefit_period,waiting_period,occupation,age,sum_assured,inc_sick_expected,inc_acc_expected,inc_count_sick,inc_count_acc,inc_count_tot,inc_amount_sick,inc_amount_acc,inc_amount_tot
,<int>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1,2018,4,m,n,a65,90d,techn,41,7119,0.000742731,0.000736533,0,0,0,0,0,0
2,2,2018,5,f,n,5yr,30d,blue,46,5582,0.001828800,0.010073520,0,0,0,0,0,0
3,3,2018,5,m,n,a65,30d,blue,41,6113,0.002475770,0.002455110,0,0,0,0,0,0
4,4,2018,3,m,n,a65,30d,prof,27,5147,0.000698100,0.000522340,0,0,0,0,0,0
5,5,2018,8,f,n,a65,30d,sed,53,8864,0.002478806,0.003137920,0,0,0,0,0,0
6,6,2018,6,f,n,2yr,14d,prof,49,11209,0.003936332,0.003655456,0,0,0,0,0,0


In [12]:
# class() returns the class of a column.
class(df$benefit_period)

[1] "character"

## Factors
From the above you'll note that the categorical columns are stored as characters. Factorising these makes them easier to work with in our models e.g. for BP factorise a65|2yr|5yr as 1|2|3. Factors are stored as integers and have labels that tell us what they are, they can be ordered and are useful for statistical analysis. 

table() returns a table of counts at each combination of column values. prop.table() converts these to a proportion. For example, applying this to the column "sex" shows us that ~75% of our data is "m" and that the other data are either "f" or "u" (unknown).

In [13]:
table(df$sex)
prop.table(table(df$sex))


     f      m      u 
120951 449487  29562 


       f        m        u 
0.201585 0.749145 0.049270 

We can then convert the columns to factors based upon the values of the column and ordering by frequency. Base level should be chosen such that it has sufficient observations for an intercept to be computed meaningfully.

In [14]:
df$sex <- factor(df$sex, levels = c("m","f","u"))
df$smoker <- factor(df$smoker, levels = c("n","s","u"))
df$benefit_period <- factor(df$benefit_period, levels = c("a65","2yr","5yr"))
df$waiting_period <- factor(df$waiting_period, labels = c("30d","14d","720d","90d"))
df$occupation <- factor(df$occupation, labels = c("prof", "sed","techn","white","blue"))

# do the same for the grouped data
df_grp$sex <- factor(df_grp$sex, levels = c("m","f","u"))
df_grp$smoker <- factor(df_grp$smoker, levels = c("n","s","u"))
df_grp$benefit_period <- factor(df_grp$benefit_period, levels = c("a65","2yr","5yr"))
df_grp$waiting_period <- factor(df_grp$waiting_period, labels = c("30d","14d","720d","90d"))
df_grp$occupation <- factor(df_grp$occupation, labels = c("prof", "sed","techn","white","blue"))

If the column is already a factor, you can extract the levels to show what order they will be used in our models

In [15]:
levels(df$sex)

[1] "m" "f" "u"

## Selection methods
table() is a method of summarizing data, returning a count at each combination of values in a column. sample() and sample_n() are other examples of selection methods. This section (not exhaustive) looks at a few more selection methods in dplyr.

In [83]:
# data subsets:  e.g. select from df where age <25 or >60 
subset(df, age <25 | age > 60) %>% head()

,id,cal_year,policy_year,sex,smoker,benefit_period,waiting_period,occupation,age,sum_assured,inc_sick_expected,inc_acc_expected,inc_count_sick,inc_count_acc,inc_count_tot,inc_amount_sick,inc_amount_acc,inc_amount_tot,imputed_age,missing_age
,<int>,<dbl>,<dbl>,<fct>,<fct>,<fct>,<fct>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
200220,220,2019,8,m,n,2yr,14d,sed,61,12171,0.014461161,0.00154438,0,0,0,0,0,0,61,0
200427,427,2019,10,f,n,a65,30d,prof,61,2279,0.002432892,0.02669789,0,0,0,0,0,0,61,0
201601,1601,2019,11,m,n,a65,14d,white,61,7299,0.014461161,0.00694971,0,0,0,0,0,0,61,0
201773,1773,2019,7,f,n,a65,14d,sed,61,10804,0.002259114,0.00423776,0,0,0,0,0,0,61,0
201927,1927,2019,9,m,n,a65,14d,prof,61,5556,0.011123970,0.00694971,0,0,0,0,0,0,61,0
202032,2032,2019,8,m,n,a65,14d,white,61,6457,0.012236367,0.00694971,0,0,0,0,0,0,61,0


In [82]:
#	dropping columns:
		# exclude columns
		mycols <- names(df) %in% c("cal_year", "smoker")
		new_df <- df[!mycols]
		# exclude 3rd and 5th column
		new_df <- df[c(-3,-5)]
		# delete columns v1 and v2 from new_df
		new_df$v3 <- new_df$v5 <- NULL
#	keeping columns: 
		# select variables v1, v2, v3
		mycols <- names(df) %in% c("cal_year", "smoker")
		new_df <- df[!mycols]
		# select 1st and 5th to 7th variables
    new_df <- df[c(1,5:7)]

,id,cal_year,policy_year,sex,smoker,benefit_period,waiting_period,occupation,age,sum_assured,inc_sick_expected,inc_acc_expected,inc_count_sick,inc_count_acc,inc_count_tot,inc_amount_sick,inc_amount_acc,inc_amount_tot,imputed_age,missing_age
,<int>,<dbl>,<dbl>,<fct>,<fct>,<fct>,<fct>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
200220,220,2019,8,m,n,2yr,14d,sed,61,12171,0.014461161,0.00154438,0,0,0,0,0,0,61,0
200427,427,2019,10,f,n,a65,30d,prof,61,2279,0.002432892,0.02669789,0,0,0,0,0,0,61,0
201601,1601,2019,11,m,n,a65,14d,white,61,7299,0.014461161,0.00694971,0,0,0,0,0,0,61,0
201773,1773,2019,7,f,n,a65,14d,sed,61,10804,0.002259114,0.00423776,0,0,0,0,0,0,61,0
201927,1927,2019,9,m,n,a65,14d,prof,61,5556,0.011123970,0.00694971,0,0,0,0,0,0,61,0
202032,2032,2019,8,m,n,a65,14d,white,61,6457,0.012236367,0.00694971,0,0,0,0,0,0,61,0


## Manipulation methods
We might want to modify our data frame to prepare it for fitting our models. The section below looks at a few simple data manipulations. Here we also introduce the infix operator (%>%); this operator passes the argument to the left of it over to the code on the right, so df %>% "operation" passes the data frame "df" over to the operation on the right.

In [17]:
# create a copy of the dataframe to work from
  new_df <- df

In [18]:
# simple manipulations
	# select as in the selection methods section, but using infix
	new_df %>% select(id, age) %>% head() # or a range using select(1:5) or select(contains("sick")) or select(starts_with("inc")); others e.g. ends_with(), last_col(), select(-age)
	# replace values in a column
	replace(new_df$sex,new_df$sex=="u","m") # no infix in base r
	# Rename, id to pol_id
	new_df %>% rename(pol_id = id) %>% head() #or (reversing the renaming)
	new_df %>% select(pol_id = id) %>% head()
	# alter data
	new_df <- new_df %>% mutate(inc_tot_expected = inc_acc_expected + inc_sick_expected) # need to assign the output back to the data frame
	# transmute - select and mutate simultaneously 
	new_df2 <- new_df %>% transmute(id, age, birth_year = cal_year - age)
	# sort
	new_df %>% arrange(desc(age)) %>% head()
	# filter
	new_df %>% filter(benefit_period == "a65", age <65) %>% head() # or
	new_df %>% filter(benefit_period %in% c("a65","5yr")) %>% head()

,id,age
,<int>,<dbl>
1,1,41
2,2,46
3,3,41
4,4,27
5,5,53
6,6,49


[1] m f m m f f m m m m m m m m f f m m f m m m m m m m m f m m f m m m m m
   [37] m m m m m m m m m f m m m m f m m m m m f m m m m f m m m m m m m m m m
   [73] m m m m m m f m m m m m m m m m m m m m m f f m f f f f m m f m f m m f
  [109] m m m f m m m m m f f m m m m m m m m m m m m m m f m m f f m f m m m m
  [145] m m m m m m m f m m m m m m m m m m m m m f f m m m m f f m m m m m f m
  [181] m m m m m m m m f f m m m m m m f m f m m f m m m m f m m m m f f f m m
  [217] m m m m f m m m m m m m m m m m f m m m m m m m m m m m m f m m m m m m
  [253] f f m m m m m m m m m m m f f f m m m m m m m m m m m f m m m m m m m m
  [289] f m m f m m m m m m m f m m m m m m m m f m m m m m m m m m m m m f m m
  [325] m m f m m f m m m f m m f m m m f m m f f f m m f f m f f m m m f f m m
  [361] m m f m m f m f f m f f m m m f f m m m m m f m m m m m m m m m f m f f
  [397] f m m m m m m f f f f m f m m f m f f m m m m m f m m m m m f m f m f m
  [433] f m f f m m m m m m m m m m m m f m m m m m m m m m m m m m m m m m f m
  [469] m f m m f m f m m m m m m m m m f m m m m m m m f f m m f m m m f m m f
  [505] m m m m f m m f m m m m m m m m f f m m f m f m m m m m m m m m m f m m
  [541] m f f m m m m m m f f m m m m f f m m f f m m f m f f f m m m m m m m m
  [577] m f m f m m m m m m m m m m m m f m m m m m m m m f m f f f m f m m m m
  [613] m f m m m m m f m m m m f m m m m f f m m m m m f m f f m m m m f m m m
  [649] f f m f m m m m m m m m m m m m m f m m f m m f m m m m m m m m m f m m
  [685] m m f m m m f m m m m m m m f m f m m m m m m m m f m f m m f m m f m m
  [721] m m m m m m m m m m m m m f f m m f m f m m m m f f m m m m m f m m m f
  [757] m m m m m m m m m m f f f m m m m m m m m m m m m m m m m m m m m f m m
  [793] m m m m m f m m m m m m m m m m m m f m m f f m f f m m m m m m m m f m
  [829] m f m m m m m m m m m m m f f m m f m f m m f m m m m m m m m m m m f m
  [865] m m m m m m m m m m m m f f f m m m m m f m m m m m m m m m f m m m f m
  [901] m f m m m m m m m f m m f m m m m m f m m m m m f m m m m m m m f f m m
  [937] m m m m f m m m f m m m m m m m m m m m m m m m f m f m m m m m m m m f
  [973] f m m f f m m m f f m f m m m m m f f m m m m m f m m f m m m m m m f m
 [1009] m m m m m m m m m f m m m f m m m m m m m f f m m m f m m m m m f f m f
 [1045] m m m m m m m f f m f m f f m m m m m m m m m f m m m m m f m m m m m f
 [1081] m m m f m m m m m m m m m m m m m f m m m m m m m m m f m m m m m m m m
 [1117] m m m m m m f m m m f m m m m m m f m m m m m m m f m m m m m f m m m m
 [1153] m m m m m m f m f m m m m m m m m f m f f m m m f m m m m f f f m m m f
 [1189] f m m m m m m m m m m m m m f m m m f m m m m m m m f m m m m m m m m m
 [1225] m m m m f m m f f m m m f m m m m m m f m m m m m m m m m m m m f m f m
 [1261] m m m m f f m m m m m m m m m m m m m m m m m f m m m m f m m f f m m f
 [1297] m m m m m m m m m m m f m f m m m m m f m m f m m m m m f m m m m m m m
 [1333] f m m m m m m m m f f m m m f m m m m f f m m m m m m m m f m f m f m m
 [1369] m m f f m m m m m m m m m m m m m m m m m m f m m m m m f m m f m m f m
 [1405] f m m m f f m m m m m m m f f m m f m m m m m m m m m m m m m m m m m m
 [1441] m m f m m m m f f m m m m f m m m f m m f m m m m m m m m m f m m m m f
 [1477] m m m m m m m m f f m m m m m m m m m m m m m f m m m m f m m m m m m m
 [1513] m m m f m m f m m m m m f m m m f m m m m m f m m f m m m m m m m f f f
 [1549] m f f m m m m m m m m f m m m f f m m m m f m m f m f m f m m f m m m m
 [1585] m m m m m m f m m m m m m m m m m m m f f m m m m m f m m m f m f m m m
 [1621] m f m m m m m m m m m m m m m m f m m m m f f m f f m m m m f m m m m m
 [1657] m f m f m f m m m m f m m f m m m m m f m m m m m m m m m m m m m m m m
 [1693] m f m m m m m m m m m m m f m m m m m m f m f m m m m m m f m m m m m f
 [1729] m m m m m m m m m m m m f m m m m m m m f m m m m m m m f f m m m m m m
 [1765] m m m m f f m f f m m m m f m m m m f m m m m f m f m m m m m m m m m m
 [18

,pol_id,cal_year,policy_year,sex,smoker,benefit_period,waiting_period,occupation,age,sum_assured,inc_sick_expected,inc_acc_expected,inc_count_sick,inc_count_acc,inc_count_tot,inc_amount_sick,inc_amount_acc,inc_amount_tot
,<int>,<dbl>,<dbl>,<fct>,<fct>,<fct>,<fct>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1,2018,4,m,n,a65,90d,white,41,7119,0.000742731,0.000736533,0,0,0,0,0,0
2,2,2018,5,f,n,5yr,14d,prof,46,5582,0.001828800,0.010073520,0,0,0,0,0,0
3,3,2018,5,m,n,a65,14d,prof,41,6113,0.002475770,0.002455110,0,0,0,0,0,0
4,4,2018,3,m,n,a65,14d,sed,27,5147,0.000698100,0.000522340,0,0,0,0,0,0
5,5,2018,8,f,n,a65,14d,techn,53,8864,0.002478806,0.003137920,0,0,0,0,0,0
6,6,2018,6,f,n,2yr,30d,sed,49,11209,0.003936332,0.003655456,0,0,0,0,0,0


,pol_id
,<int>
1,1
2,2
3,3
4,4
5,5
6,6


,id,cal_year,policy_year,sex,smoker,benefit_period,waiting_period,occupation,age,sum_assured,inc_sick_expected,inc_acc_expected,inc_count_sick,inc_count_acc,inc_count_tot,inc_amount_sick,inc_amount_acc,inc_amount_tot,inc_tot_expected
,<int>,<dbl>,<dbl>,<fct>,<fct>,<fct>,<fct>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,220,2020,9,m,n,2yr,14d,sed,62,12171,0.014957228,0.00160944,0,0,0,0,0,0,0.016566668
2,427,2020,11,f,n,a65,30d,prof,62,2279,0.002617888,0.02756174,0,0,0,0,0,0,0.030179632
3,1601,2020,12,m,n,a65,14d,white,62,7299,0.014957228,0.00724248,0,0,0,0,0,0,0.022199708
4,1773,2020,8,f,n,a65,14d,sed,62,10804,0.002056912,0.00437488,0,0,0,0,0,0,0.006431792
5,1927,2020,10,m,n,a65,14d,prof,62,5556,0.011505560,0.00724248,0,0,0,0,0,0,0.018748040
6,2032,2020,9,m,n,a65,14d,white,62,6457,0.012656116,0.00724248,0,0,0,0,0,0,0.019898596


,id,cal_year,policy_year,sex,smoker,benefit_period,waiting_period,occupation,age,sum_assured,inc_sick_expected,inc_acc_expected,inc_count_sick,inc_count_acc,inc_count_tot,inc_amount_sick,inc_amount_acc,inc_amount_tot,inc_tot_expected
,<int>,<dbl>,<dbl>,<fct>,<fct>,<fct>,<fct>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1,2018,4,m,n,a65,90d,white,41,7119,0.000742731,0.000736533,0,0,0,0,0,0,0.001479264
2,3,2018,5,m,n,a65,14d,prof,41,6113,0.002475770,0.002455110,0,0,0,0,0,0,0.004930880
3,4,2018,3,m,n,a65,14d,sed,27,5147,0.000698100,0.000522340,0,0,0,0,0,0,0.001220440
4,5,2018,8,f,n,a65,14d,techn,53,8864,0.002478806,0.003137920,0,0,0,0,0,0,0.005616726
5,7,2018,6,m,n,a65,720d,white,54,6378,0.001776958,0.001003248,0,0,0,0,0,0,0.002780206
6,8,2018,6,m,n,a65,14d,sed,42,6780,0.003111636,0.000574240,0,0,0,0,0,0,0.003685876


,id,cal_year,policy_year,sex,smoker,benefit_period,waiting_period,occupation,age,sum_assured,inc_sick_expected,inc_acc_expected,inc_count_sick,inc_count_acc,inc_count_tot,inc_amount_sick,inc_amount_acc,inc_amount_tot,inc_tot_expected
,<int>,<dbl>,<dbl>,<fct>,<fct>,<fct>,<fct>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1,2018,4,m,n,a65,90d,white,41,7119,0.000742731,0.000736533,0,0,0,0,0,0,0.001479264
2,2,2018,5,f,n,5yr,14d,prof,46,5582,0.001828800,0.010073520,0,0,0,0,0,0,0.011902320
3,3,2018,5,m,n,a65,14d,prof,41,6113,0.002475770,0.002455110,0,0,0,0,0,0,0.004930880
4,4,2018,3,m,n,a65,14d,sed,27,5147,0.000698100,0.000522340,0,0,0,0,0,0,0.001220440
5,5,2018,8,f,n,a65,14d,techn,53,8864,0.002478806,0.003137920,0,0,0,0,0,0,0.005616726
6,7,2018,6,m,n,a65,720d,white,54,6378,0.001776958,0.001003248,0,0,0,0,0,0,0.002780206


In [19]:
# aggregations
	# group by, also ungroup()
	new_df %>% group_by(sex) %>% # can add a mutate to group by which will aggregate only to the level specified in the group_by e.g. 
	mutate(sa_by_sex = sum(sum_assured)) %>% head() # adds a new column with the total sum assured by sex.
	# after doing this, ungroup() in order to apply future operations to all records individually
	# count, sorting by most frequent and weighting by another column
	new_df %>% count(sex, wt= sum_assured, sort=TRUE)  # counts the number of entries for each value of sex, weighted by sum assured
	# summarize takes many observations and turns them into one observation. mean(), median(), min(), max(), and n() for the size of the group
	new_df %>% summarize(total = sum(sum_assured), min_age = min(age), max_age = max(age), max(inc_tot_expected)) 
	new_df %>% group_by(sex) %>% summarise(n = n())
	table(new_df$sex) # returns count by sex; no infix in base r

id,cal_year,policy_year,sex,smoker,benefit_period,waiting_period,occupation,age,sum_assured,inc_sick_expected,inc_acc_expected,inc_count_sick,inc_count_acc,inc_count_tot,inc_amount_sick,inc_amount_acc,inc_amount_tot,inc_tot_expected,sa_by_sex
<int>,<dbl>,<dbl>,<fct>,<fct>,<fct>,<fct>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,2018,4,m,n,a65,90d,white,41,7119,0.000742731,0.000736533,0,0,0,0,0,0,0.001479264,3198326349
2,2018,5,f,n,5yr,14d,prof,46,5582,0.001828800,0.010073520,0,0,0,0,0,0,0.011902320,860378043
3,2018,5,m,n,a65,14d,prof,41,6113,0.002475770,0.002455110,0,0,0,0,0,0,0.004930880,3198326349
4,2018,3,m,n,a65,14d,sed,27,5147,0.000698100,0.000522340,0,0,0,0,0,0,0.001220440,3198326349
5,2018,8,f,n,a65,14d,techn,53,8864,0.002478806,0.003137920,0,0,0,0,0,0,0.005616726,860378043
6,2018,6,f,n,2yr,30d,sed,49,11209,0.003936332,0.003655456,0,0,0,0,0,0,0.007591788,860378043


sex,n
<fct>,<dbl>
m,3198326349
f,860378043
u,210196386


total,min_age,max_age,max(inc_tot_expected)
<dbl>,<dbl>,<dbl>,<dbl>
4268900778,25,62,0.04079533


sex,n
<fct>,<int>
m,449487
f,120951
u,29562



     m      f      u 
449487 120951  29562 

In [20]:
# outliers
	new_df %>% top_n(10, inc_tot_expected) # also operates on grouped table - returns top n per group

id,cal_year,policy_year,sex,smoker,benefit_period,waiting_period,occupation,age,sum_assured,inc_sick_expected,inc_acc_expected,inc_count_sick,inc_count_acc,inc_count_tot,inc_amount_sick,inc_amount_acc,inc_amount_tot,inc_tot_expected
<int>,<dbl>,<dbl>,<fct>,<fct>,<fct>,<fct>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
15563,2018,9,f,s,2yr,30d,white,56,7432,0.004894915,0.03240513,0,0,0,0,0,0,0.03730005
173520,2018,6,f,s,a65,30d,white,55,7687,0.004974007,0.03115035,0,0,0,0,0,0,0.03612436
15563,2019,10,f,s,2yr,30d,white,57,7432,0.004778486,0.03366503,0,0,0,0,0,0,0.03844352
115542,2019,11,m,s,a65,30d,white,60,4581,0.025730250,0.01351980,0,0,0,0,0,0,0.03925005
173520,2019,7,f,s,a65,30d,white,56,7687,0.004894915,0.03240513,0,0,0,0,0,0,0.03730005
188978,2019,8,m,s,a65,30d,white,61,4006,0.022581659,0.01410791,0,0,0,0,0,0,0.03668957
15563,2020,11,f,s,2yr,30d,white,58,7432,0.005462097,0.03492786,0,0,0,0,0,0,0.04038995
115542,2020,12,m,s,a65,30d,white,61,4581,0.026687415,0.01410791,0,0,0,0,0,0,0.04079533
173520,2020,8,f,s,a65,30d,white,57,7687,0.004778486,0.03366503,0,0,0,0,0,0,0.03844352


Looking at total count of claim rather than just sickness shows a slight trend by age: